In [1]:
import os
import pandas as pd
import zipfile
import csv
from io import StringIO

#### For Windows

In [2]:
directory = 'C:\\Users\\aport\\OneDrive\\Documents\\School\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips_small\\'

In [3]:
directory + 'transArchive_201001_201003_small.zip'

'C:\\Users\\aport\\OneDrive\\Documents\\School\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips_small\\transArchive_201001_201003_small.zip'

#### For Mac

In [11]:
directory = '/Users/aaronportra/Documents/Advanced Data Analytics/wedge_project/WedgeZipOfZips_small/'

In [5]:
directory + 'transArchive_201001_201003_small.zip'

'C:\\Users\\aport\\OneDrive\\Documents\\School\\Applied Data Analytics\\wedge_project\\WedgeZipOfZips_small\\transArchive_201001_201003_small.zip'

## Cleaning Data for BQ

In [ ]:
file = zipfile.ZipFile(directory)

In [ ]:
for file in os.listdir(directory):
    if file.endswith('.zip'):
        with zipfile.ZipFile(directory + file,'r') as zip_file:
            for info in zip_file.namelist():

                file_content = zip_file.read(info).decode('utf-8')

                csv_file = StringIO(file_content)

                print(pd.read_csv(csv_file))

In [42]:
i = 0
for name in os.listdir(directory):
    d = zipfile.ZipFile(directory + name)
    for content in d.infolist():
        print(content)
        i +=1
print(i)

<ZipInfo filename='transArchive_201001_201003_small.csv' filemode='-rw-rw-rw-' file_size=2948918>
<ZipInfo filename='transArchive_201004_201006_small.csv' filemode='-rw-rw-rw-' file_size=2910071>
<ZipInfo filename='transArchive_201007_201009_small.csv' filemode='-rw-rw-rw-' file_size=2908093>
<ZipInfo filename='transArchive_201010_201012_small.csv' filemode='-rw-rw-rw-' file_size=2936765>
<ZipInfo filename='transArchive_201101_201103_small.csv' filemode='-rw-rw-rw-' file_size=2970096>
<ZipInfo filename='transArchive_201104_small.csv' filemode='-rw-rw-rw-' file_size=2147765>
<ZipInfo filename='transArchive_201105_small.csv' filemode='-rw-rw-rw-' file_size=2163264>
<ZipInfo filename='transArchive_201106_small.csv' filemode='-rw-rw-rw-' file_size=2132916>
<ZipInfo filename='transArchive_201107_201109_small.csv' filemode='-rw-rw-rw-' file_size=2960648>
<ZipInfo filename='transArchive_201110_201112_small.csv' filemode='-rw-rw-rw-' file_size=2972362>
<ZipInfo filename='transArchive_201201_20

In [33]:
for path, folders, files in os.walk(directory):
    # Open file
    for filename in files:
        with open(os.path.join(directory, filename)) as f:
            print(f"Content of '{filename}'")
            # Read content of file


    # List contain of folder
    for folder_name in folders:
        print(f"Content of '{folder_name}'")
        # List content from folder
        print(os.listdir(f"{path}/{folder_name}"))
        print()

    break

Content of 'transArchive_201001_201003_small.zip'
Content of 'transArchive_201004_201006_small.zip'
Content of 'transArchive_201007_201009_small.zip'
Content of 'transArchive_201010_201012_small.zip'
Content of 'transArchive_201101_201103_small.zip'
Content of 'transArchive_201104_small.zip'
Content of 'transArchive_201105_small.zip'
Content of 'transArchive_201106_small.zip'
Content of 'transArchive_201107_201109_small.zip'
Content of 'transArchive_201110_201112_small.zip'
Content of 'transArchive_201201_201203_inactive_small.zip'
Content of 'transArchive_201201_201203_small.zip'
Content of 'transArchive_201204_201206_inactive_small.zip'
Content of 'transArchive_201204_201206_small.zip'
Content of 'transArchive_201207_201209_inactive_small.zip'
Content of 'transArchive_201207_201209_small.zip'
Content of 'transArchive_201210_201212_inactive_small.zip'
Content of 'transArchive_201210_201212_small.zip'
Content of 'transArchive_201301_201303_inactive_small.zip'
Content of 'transArchive_2